# Multi-Agent Conversations and Emergent Collaboration

> Computational Analysis of Social Complexity
>
> Fall 2025, Spencer Lyon

**Prerequisites**

- L.A1.01 (LLM basics and API calls)
- L.A1.02 (RAG systems)
- Game Theory basics (Week 8)
- Agent-Based Models (Week 6-7)

**Outcomes**

- Design multi-agent conversation systems with role-based agents
- Implement agent coordination patterns in Julia
- Analyze emergent behaviors in AI agent groups
- Apply game-theoretic concepts to AI agent interactions
- Build a disaster response simulation with specialized agents

**References**

- [AutoGen: Enabling Next-Gen LLM Applications](https://microsoft.github.io/autogen/)
- [Multi-Agent Systems: Survey](https://arxiv.org/abs/2401.03428)
- [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442)
- [Anthropic: Constitutional AI](https://www.anthropic.com/index/constitutional-ai-harmlessness-from-ai-feedback)

## Introduction

- So far we've explored individual AI agents and how to augment them with external knowledge
- But many of the most interesting complex systems emerge from **interactions** between multiple entities
- Recall from Week 6-7: the Schelling model showed how simple local interactions create surprising aggregate patterns
- From Week 8: game theory taught us that outcomes depend not just on individual actions, but on strategic interactions
- Today we extend these ideas to AI agents: what happens when multiple LLM-powered agents communicate and collaborate?
- We'll discover that **emergent collaboration** can solve problems that single agents cannot

## From Single Agents to Multi-Agent Systems

### The Limits of Individual Agents

- A single LLM agent, no matter how sophisticated, faces fundamental constraints:
  - **Context window limits**: Can only "remember" the last N tokens
  - **Single perspective**: One prompt, one approach, one set of biases
  - **No self-correction**: Limited ability to catch its own mistakes
  - **Generalist burden**: Must be good at everything simultaneously

### Multi-Agent Advantages

- Multiple agents working together can overcome these limitations:
  - **Specialization**: Different agents for different tasks (like division of labor in economics)
  - **Diverse perspectives**: Multiple approaches to the same problem
  - **Error correction**: Agents can critique and improve each other's outputs
  - **Scalability**: Distribute work across parallel agents

### Connection to Course Themes

- **ABMs**: Like Schelling agents, AI agents have simple rules (their prompts) but complex interactions
- **Networks**: Agents form communication networks with different topologies
- **Game Theory**: Strategic interaction between agents pursuing different objectives
- **Emergence**: Coordination patterns emerge without central planning

## Agent Communication Protocols

### Message Passing Architecture

- How do AI agents communicate?
- In traditional ABMs (like Schelling), agents interact through shared environment state
- In AI multi-agent systems, agents typically use **explicit message passing**
- Each message contains:
  - **Sender**: Which agent created the message
  - **Content**: The actual text/data
  - **Metadata**: Timestamp, message type, intended recipient(s)



### Conversation Structure

- We need to decide on a conversation flow:
  1. **Sequential**: Agents take turns in fixed order (round-robin)
  2. **Dynamic**: Next speaker chosen based on context or rules
  3. **Broadcast**: One agent speaks to all others
  4. **Selective**: Agents choose who to respond to

- For today, we'll focus on sequential and dynamic patterns

## Building Blocks: Julia Implementation

- Let's build our multi-agent framework from first principles
- We'll leverage Julia's type system to create clean abstractions
- First, we need to load our libraries

In [1]:
using HTTP
using JSON3
using DataFrames
using Dates

using DotEnv

DotEnv.load!()

### Message Type

- We'll start by defining a `Message` type to represent communications between agents
- This is similar to how we defined agent types in Week 6

In [2]:
struct Message
    sender::String
    content::String
    timestamp::DateTime
    metadata::Dict{String, Any}
end

# Constructor with defaults
function Message(sender::String, content::String; metadata=Dict{String, Any}())
    Message(sender, content, now(), metadata)
end

Message

### Agent Type

- Now we define our AI Agent
- Unlike Schelling agents (which had fixed rules), our agents are defined by:
  - **Role**: What perspective or specialty they have
  - **System prompt**: Instructions that shape their behavior
  - **Model**: Which LLM they use (we'll default to Claude)
  - **Memory**: Their conversation history

In [3]:
mutable struct AIAgent
    role::String
    system_prompt::String
    model::String
    memory::Vector{Message}
    api_key::String
end

# Constructor
function AIAgent(role::String, system_prompt::String;
                 model="claude-haiku-4-5",
                 api_key=get(ENV, "ANTHROPIC_API_KEY", ""))
    AIAgent(role, system_prompt, model, Message[], api_key)
end

# Custom display method to hide API key
function Base.show(io::IO, agent::AIAgent)
    masked_key = length(agent.api_key) > 8 ? "***" * agent.api_key[end-4:end] : "***"
    prompt_preview = length(agent.system_prompt) > 50 ?
        agent.system_prompt[1:50] * "..." : agent.system_prompt
    print(io, "AIAgent(role=\"$(agent.role)\", ",
          "system_prompt=\"$(prompt_preview)\", ",
          "model=\"$(agent.model)\", ",
          "memory=$(length(agent.memory)) messages, ",
          "api_key=\"$masked_key\")")
end

### Agent Response Function

- This function sends a message to an agent and gets its response
- It's similar to the API call from L.A1.01, but now includes conversation history

In [4]:
function get_response(agent::AIAgent, new_message::String)::String
    # Build conversation history for API
    messages = []

    # Add history
    for msg in agent.memory
        role = msg.sender == agent.role ? "assistant" : "user"
        push!(messages, Dict("role" => role, "content" => msg.content))
    end

    # Add new message
    push!(messages, Dict("role" => "user", "content" => new_message))

    # Prepare API request
    headers = [
        "x-api-key" => agent.api_key,
        "anthropic-version" => "2023-06-01",
        "content-type" => "application/json"
    ]

    body = Dict(
        "model" => agent.model,
        "max_tokens" => 1024,
        "system" => agent.system_prompt,
        "messages" => messages
    )

    # Make API call
    response = HTTP.post(
        "https://api.anthropic.com/v1/messages",
        headers,
        JSON3.write(body)
    )

    # Parse response
    result = JSON3.read(response.body)
    content = result.content[1].text

    # Store in memory
    push!(agent.memory, Message("user", new_message))
    push!(agent.memory, Message(agent.role, content))

    return content
end

get_response (generic function with 1 method)

## Simple Two-Agent Conversation

- Let's start with the simplest multi-agent system: two agents having a conversation
- We'll create a **Researcher** and a **Critic**
- This mirrors the "debate" format common in multi-agent systems

### Creating Our Agents

In [5]:
# Create researcher agent
researcher = AIAgent(
    "Researcher",
    """You are a researcher proposing hypotheses about social phenomena.
    Your job is to generate creative, testable ideas about how social networks
    and human behavior work. Be bold and specific in your proposals.
    Keep responses to 2-3 sentences."""
)

# Create critic agent
critic = AIAgent(
    "Critic",
    """You are a critical reviewer who evaluates research hypotheses.
    Your job is to identify weaknesses, suggest improvements, and ask
    probing questions. Be constructive but rigorous.
    Keep responses to 2-3 sentences."""
)

AIAgent(role="Critic", system_prompt="You are a critical reviewer who evaluates research...", model="claude-haiku-4-5", memory=0 messages, api_key="***L0gAA")

### Running a Conversation

- Now we'll orchestrate a multi-turn conversation
- The researcher proposes ideas, the critic responds, and this continues for several rounds

In [6]:
function run_conversation(agent1::AIAgent, agent2::AIAgent,
                          initial_prompt::String, turns::Int=3)
    # Start with initial prompt to first agent
    current_message = initial_prompt

    println("="^60)
    println("Starting conversation...\n")

    for turn in 1:turns
        # Agent 1 responds
        println("\n[$(agent1.role)]")
        response1 = get_response(agent1, current_message)
        println(response1)

        # Agent 2 responds
        println("\n[$(agent2.role)]")
        response2 = get_response(agent2, response1)
        println(response2)

        current_message = response2
        println("\n" * "-"^60)
    end
end

run_conversation (generic function with 2 methods)

In [7]:
# Run the conversation
run_conversation(
    researcher,
    critic,
    "Propose a hypothesis about how weak ties in social networks affect information spread.",
    3
)

Starting conversation...


[Researcher]
**Hypothesis**: Weak ties are disproportionately responsible for transmitting novel information across network clusters, so individuals who strategically maintain bridges to distant social groups adopt innovations 2-3x faster than those embedded in tight-knit communities—but this advantage disappears for information requiring deep trust or behavioral change.

**Testable prediction**: Track adoption timing of a new workplace tool across employees with varying weak-tie density; we'd expect early adopters to have significantly more connections outside their immediate department, while late adopters (6+ months later) show no correlation with tie diversity.

[Critic]
**Strengths**: You're threading together network theory elegantly—the caveat about trust-dependent information is sophisticated and prevents oversimplification.

**Critical weaknesses**:

1. **Confounding variables undermine causality**: Early adopters likely differ in tech-savviness, org

### What Did We Observe?

- The researcher proposed ideas
- The critic identified weaknesses and suggested improvements
- Ideas evolved over multiple rounds of interaction
- This is **emergent refinement** - neither agent could produce the final result alone
- Connection to game theory: this is like a **cooperative game** where both agents benefit from collaboration

## Multi-Agent Coordination Patterns

### AutoGen-Style Architecture

- Microsoft's AutoGen framework introduced several powerful patterns for multi-agent systems
- Key idea: **role-based specialization** with structured handoffs
- Common roles:
  - **User proxy**: Represents human input and approvals
  - **Assistant**: General problem solver
  - **Executor**: Runs code and reports results
  - **Critic**: Reviews and validates outputs

### Sequential vs. Graph-Based Coordination

- **Sequential**: A → B → C → D (like an assembly line)
  - Simple to implement and reason about
  - Limited flexibility
  
- **Graph-based**: Agents can call each other based on conditions
  - More flexible and powerful
  - Can create cycles (debate loops) or branches (conditional paths)
  - Requires careful design to avoid infinite loops

### Connection to Networks

- Multi-agent coordination is a **directed graph**!
- Nodes = agents, Edges = possible communications
- Topology affects performance:
  - **Chain**: Sequential processing
  - **Star**: Central coordinator (hub)
  - **Fully connected**: Any agent can talk to any other
  - **Hierarchical**: Manager-worker structure

## Case Study: Disaster Response Team

- Let's build a more complex system inspired by real-world needs
- Context: After Hurricane Ian, Google's Area 120 team explored using AI for disaster response
- We'll simulate a disaster response team with multiple specialized agents
- This demonstrates how multi-agent systems can handle complex, time-critical scenarios

### The Scenario

- A major hurricane has just made landfall
- Multiple communities need coordinated response
- Resources are limited and must be allocated efficiently
- Our AI agents will:
  - Assess the situation from different perspectives
  - Coordinate resource allocation
  - Adapt to new information
  - Make decisions without centralized control

### Defining Specialized Agents

- We'll create four specialized agents:
  1. **Logistics Coordinator**: Manages supplies and transportation
  2. **Medical Coordinator**: Prioritizes health and safety
  3. **Communications Officer**: Handles public information
  4. **Situation Analyst**: Synthesizes information and recommends actions

In [8]:
# Create disaster response team
logistics = AIAgent(
    "Logistics",
    """You are a logistics coordinator for disaster response.
    Your role is to manage supply chains, transportation, and resource allocation.
    Consider: road conditions, fuel availability, warehouse locations, delivery times.
    Be specific about quantities and locations. Keep responses concise (3-4 sentences)."""
)

medical = AIAgent(
    "Medical",
    """You are a medical coordinator for disaster response.
    Your role is to prioritize health and safety needs, triage resources.
    Consider: injuries, disease risk, medication needs, vulnerable populations.
    Advocate for medical priorities even if expensive. Keep responses concise (3-4 sentences)."""
)

comms = AIAgent(
    "Communications",
    """You are a communications officer for disaster response.
    Your role is to manage public information and coordinate between teams.
    Consider: panic prevention, accurate information, accessibility, multiple languages.
    Summarize key points for public consumption. Keep responses concise (3-4 sentences)."""
)

analyst = AIAgent(
    "Analyst",
    """You are a situation analyst for disaster response.
    Your role is to synthesize information from all sources and recommend actions.
    Consider: tradeoffs, priorities, uncertainties, time constraints.
    Provide clear recommendations with reasoning. Keep responses concise (3-4 sentences)."""
)

AIAgent(role="Analyst", system_prompt="You are a situation analyst for disaster response....", model="claude-haiku-4-5", memory=0 messages, api_key="***L0gAA")

### Multi-Agent Response Protocol

- We'll implement a coordination protocol:
  1. **Initial assessment**: Each specialist evaluates the situation independently
  2. **Discussion phase**: Specialists share perspectives and debate priorities
  3. **Synthesis**: Analyst integrates inputs and proposes action plan
  4. **Validation**: Team reviews and refines the plan

- This mirrors real incident command structures but without rigid hierarchy

In [14]:
function disaster_response_round(scenario::String, team::Vector{AIAgent})
    println("\n" * "="^70)
    println("DISASTER SCENARIO")
    println("="^70)
    println(scenario)
    println()

    # Phase 1: Independent assessments
    println("\n" * "="^70)
    println("PHASE 1: INDEPENDENT ASSESSMENTS")
    println("="^70)

    assessments = Dict{String, String}()
    for agent in team[1:end-1]  # All except analyst
        println("\n[$(agent.role) Assessment]")
        assessment = get_response(agent, "Given this scenario: $scenario\n\nProvide your assessment from your area of expertise.")
        println(assessment)
        assessments[agent.role] = assessment
    end

    # Phase 2: Synthesis
    println("\n" * "="^70)
    println("PHASE 2: SITUATION ANALYSIS AND RECOMMENDATIONS")
    println("="^70)

    # Build summary for analyst
    summary = "Team assessments:\n\n"
    for (role, assessment) in assessments
        summary *= "$role: $assessment\n\n"
    end
    summary *= "Based on these inputs, what are your top 3 recommended immediate actions?"

    println("\n[Analyst Recommendations]")
    recommendations = get_response(team[end], summary)
    println(recommendations)

    # Phase 3: Quick validation round
    println("\n" * "="^70)
    println("PHASE 3: TEAM VALIDATION")
    println("="^70)

    for agent in team[1:end-1]
        println("\n[$(agent.role) Response]")
        validation = get_response(agent, "The analyst recommends: $recommendations\n\nBriefly respond: any critical concerns or support?")
        println(validation)
    end

    return recommendations
end

disaster_response_round (generic function with 1 method)

### Running the Simulation

In [15]:
scenario1 = """
Hurricane Zeta made landfall 6 hours ago as a Category 4 storm.
Affected areas: Coastal City (pop. 50,000), Rural County (pop. 15,000), Island Community (pop. 3,000).

Current situation:
- Coastal City: 70% without power, flooding in downtown, hospital on backup generator
- Rural County: Main highway bridge damaged, several farms isolated, nursing home needs evacuation
- Island Community: Complete power loss, bridge to mainland impassable, limited supplies

Available resources:
- 10 helicopters (limited by weather)
- 50 trucks with supplies (food, water, medical)
- 200 personnel (EMTs, engineers, volunteers)
- 2 mobile hospitals

Constraints: Storm surge warnings continue for 12 hours. Next weather window for air operations: 4 hours.
"""

team = [logistics, medical, comms, analyst]
plan = disaster_response_round(scenario1, team)


DISASTER SCENARIO
Hurricane Zeta made landfall 6 hours ago as a Category 4 storm.
Affected areas: Coastal City (pop. 50,000), Rural County (pop. 15,000), Island Community (pop. 3,000).

Current situation:
- Coastal City: 70% without power, flooding in downtown, hospital on backup generator
- Rural County: Main highway bridge damaged, several farms isolated, nursing home needs evacuation
- Island Community: Complete power loss, bridge to mainland impassable, limited supplies

Available resources:
- 10 helicopters (limited by weather)
- 50 trucks with supplies (food, water, medical)
- 200 personnel (EMTs, engineers, volunteers)
- 2 mobile hospitals

Constraints: Storm surge warnings continue for 12 hours. Next weather window for air operations: 4 hours.



PHASE 1: INDEPENDENT ASSESSMENTS

[Logistics Assessment]
# Immediate Logistics Assessment (Next 12 Hours)

**Priority 1 - Island Community (4-hour window):**
Deploy 3 helicopters now to evacuate nursing home patients and critical case

"# TOP 3 IMMEDIATE ACTIONS (Next 4 Hours)\n\n## 1. **Launch Island Community Helicopter Operations NOW (3 helicopters)**\nExecute dual mission: evacuate nursing home patients and critical cases on outbound flights; shuttle 2 tons of supplies + medical team on return flights" ⋯ 1551 bytes ⋯ "rastructure. All three areas are simultaneously addressed within our capacity constraints.\n\n---\n\n**Critical Watch:** Medical and Logistics teams coordinate Island evacuation manifest in real-time so Communications can provide families accurate ETA updates every 2 hours."

### Analyzing the Emergent Behavior

- What did we observe?
  - **Specialization**: Each agent focused on their domain expertise
  - **Disagreement**: Agents had different priorities (medical urgency vs. logistical feasibility)
  - **Negotiation**: Implicit through the discussion and validation phases
  - **Consensus formation**: Final plan incorporated multiple perspectives

- This is **emergent coordination** without central command!
- No single agent had authority, yet a coherent plan emerged
- Connection to game theory: This is a **cooperative game with communication**

## Game-Theoretic Analysis of Multi-Agent Systems

### From ABMs to Strategic Agents

- Recall the Schelling model: agents had **fixed** preference functions
- AI agents are fundamentally different: they can **reason** about others' behavior
- This makes them strategic players in the game-theoretic sense

### Coordination Games

- Multi-agent AI systems often face **coordination problems**
- Example: Which disaster area to prioritize?
  - If both medical and logistics focus on Island Community → waste resources
  - If they split effort optimally → better outcomes
  - Multiple equilibria possible (each area could be prioritized)

- Communication helps select among equilibria
- This is why our multi-agent protocol included explicit discussion phases

### Mechanism Design for AI Agents

- Question: How do we design prompts and protocols to achieve desired outcomes?
- This is **mechanism design** applied to AI systems
- Key considerations:
  1. **Incentive compatibility**: Each agent's prompt should make truthful reporting optimal
  2. **Information aggregation**: How do we combine diverse perspectives?
  3. **Termination**: How do we ensure conversations conclude productively?

### Example: Voting vs. Consensus

- Two ways to make group decisions:
  - **Voting**: Each agent proposes action, majority wins (simple but can ignore minorities)
  - **Consensus**: Discussion until agreement (thorough but can be slow)
  
- Our disaster protocol used a hybrid: discussion + analyst synthesis
- This is a **dictator game** where the analyst has final say, but is informed by others

## Implementing Voting and Consensus Mechanisms

### Voting Mechanism

- Let's implement a simple voting system for multi-agent decision making
- Each agent votes for an option, and we tally results

In [18]:
function voting_decision(agents::Vector{AIAgent}, question::String, options::Vector{String})
    println("\n" * "="^70)
    println("VOTING ROUND")
    println("="^70)
    println("Question: $question")
    println("Options: ", join(options, ", "))
    println()

    votes = Dict{String, Int}()
    for opt in options
        votes[opt] = 0
    end

    vote_prompt = """$question

    Options: $(join(options, ", "))

    Vote for ONE option and briefly explain why (1 sentence).
    Format your response as: VOTE: [option] - [reason]"""

    for agent in agents
        println("\n[$(agent.role)]")
        response = get_response(agent, vote_prompt)
        println(response)

        # Parse vote (simple string matching)
        for opt in options
            # Look for "VOTE:" followed by the option
            vote_match = match(r"VOTE:\s*([^-\n]+)", response)
            if vote_match !== nothing
                vote_text = strip(vote_match.captures[1])
                if occursin(opt, vote_text)
                    votes[opt] += 1
                    break
                end
            end
        end
    end

    println("\n" * "="^70)
    println("RESULTS")
    println("="^70)
    for (option, count) in sort(collect(votes), by=x->x[2], rev=true)
        println("$option: $count votes")
    end

    winner = argmax(votes)
    println("\nDecision: $winner")
    return winner
end

voting_decision (generic function with 1 method)

In [19]:
# Example: Prioritize which area to help first
decision = voting_decision(
    [logistics, medical, comms],
    "Which area should receive the first wave of helicopter support?",
    ["Coastal City", "Rural County", "Island Community"]
)


VOTING ROUND
Question: Which area should receive the first wave of helicopter support?
Options: Coastal City, Rural County, Island Community


[Logistics]
VOTE: Island Community - It's the only area where helicopter support is time-critical and irreplaceable; Coastal City and Rural County have ground-based alternatives, but the Island loses all access in 4 hours once the weather window closes.

[Medical]
VOTE: Island Community - it's the only area where helicopters are the sole lifeline; the 4-hour weather window closes permanently, making this our only opportunity to reach medically vulnerable populations before 24+ hours of isolation.

[Communications]
VOTE: Island Community - Complete isolation (impassable bridge), total power loss, and vulnerable population (nursing home) create irreversible life-safety risk if we miss the 4-hour weather window; the other two areas have ground-based alternatives.

RESULTS
Island Community: 3 votes
Rural County: 0 votes
Coastal City: 0 votes

Decis

"Island Community"

### Consensus Through Debate

- Voting is fast but doesn't capture nuance
- Sometimes we need **consensus** - a decision everyone can support
- Let's implement a debate-to-consensus mechanism

In [20]:
function consensus_decision(agents::Vector{AIAgent}, question::String, max_rounds::Int=3)
    println("\n" * "="^70)
    println("CONSENSUS BUILDING")
    println("="^70)
    println("Question: $question\n")

    # Initial proposals
    println("ROUND 1: Initial Proposals")
    println("-"^70)
    proposals = String[]
    for agent in agents
        println("\n[$(agent.role)]")
        proposal = get_response(agent, "$question\n\nProvide your initial proposal and reasoning.")
        println(proposal)
        push!(proposals, proposal)
    end

    # Debate rounds
    for round in 2:max_rounds
        println("\n" * "="^70)
        println("ROUND $round: Discussion and Refinement")
        println("-"^70)

        summary = "Previous proposals:\n" * join(["$(agents[i].role): $(proposals[i])" for i in 1:length(agents)], "\n\n")

        new_proposals = String[]
        for agent in agents
            println("\n[$(agent.role)]")
            response = get_response(agent, "$summary\n\nGiven these proposals, refine your position or indicate agreement with another proposal.")
            println(response)
            push!(new_proposals, response)
        end
        proposals = new_proposals
    end

    # Final consensus check
    println("\n" * "="^70)
    println("FINAL CONSENSUS")
    println("="^70)
    final_summary = "After discussion, the proposals are:\n" * join(["$(agents[i].role): $(proposals[i])" for i in 1:length(agents)], "\n\n")
    println(final_summary)

    return proposals
end

consensus_decision (generic function with 2 methods)

In [21]:
# Example: Build consensus on evacuation priority
consensus = consensus_decision(
    [logistics, medical],
    "What should be the priority order for evacuating the nursing home in Rural County?",
    2
)


CONSENSUS BUILDING
Question: What should be the priority order for evacuating the nursing home in Rural County?

ROUND 1: Initial Proposals
----------------------------------------------------------------------

[Logistics]
# Rural County Nursing Home Evacuation Priority Order

**Proposed Sequence:**
1. **Tier 1 (First):** Ambulatory residents + staff (fastest throughput, frees capacity)
2. **Tier 2 (Second):** Bedridden/mobility-limited residents requiring 1:1 assistance
3. **Tier 3 (Final):** Residents on life support or requiring continuous medical monitoring (load last but depart first with dedicated medical personnel)

**Reasoning:** 
Tier 1 evacuates quickly using the mobile hospital's transport capacity efficiently; Tier 2 requires more time per person but is medically stable; Tier 3 loads last but departs with full medical escort to ensure continuous care during transport—this prevents abandoning critical patients and maximizes vehicle utilization. The convoy should stage Tier

2-element Vector{String}:
 "# Refined Position: AGREEMENT " ⋯ 1342 bytes ⋯ "s primary evacuation sequence."
 "# REFINED POSITION: Hybrid App" ⋯ 1377 bytes ⋯ "sals work *together*.\n\nConcur?"

## Emergent Behavior Analysis

### What Makes Behavior "Emergent"?

- Recall from the ABM money lecture that we didn't specify that wealth followed a power law: that result *emerged*
**Emergence** means system-level patterns not explicitly programmed
- In Schelling model: segregation emerged from mild individual preferences
- In our multi-agent system:
  - **No agent knows the "correct" answer**
  - **No central controller** dictating decisions
  - **Coordination emerges** from local interactions (conversations)
  - **Better solutions** than any individual agent could produce

### Comparing to Traditional ABMs

| Traditional ABM (Schelling) | AI Multi-Agent System |
|----------------------------|----------------------|
| Fixed rules | Learned behaviors |
| No reasoning | Explicit reasoning |
| Implicit communication (environment) | Explicit communication (messages) |
| Simple agents, complex patterns | Complex agents, complex patterns |
| Deterministic (given rules) | Stochastic (LLM sampling) |

### Sources of Emergence in AI Systems

1. **Prompt interpretation**: Each agent interprets prompts slightly differently
2. **Stochastic sampling**: LLMs don't give identical outputs every time
3. **Conversation dynamics**: Order of speakers affects outcomes
4. **Information aggregation**: Synthesis creates novel insights
5. **Feedback loops**: Later responses build on earlier ones

## Connection to Collective Intelligence

### Wisdom of Crowds

- Classic result (Surowiecki, 2004): aggregated judgments often beat individual experts
- Requires:
  1. **Diversity**: Different perspectives and information
  2. **Independence**: Agents form opinions without undue influence
  3. **Decentralization**: Local knowledge utilized
  4. **Aggregation**: Mechanism to combine judgments

- Our multi-agent systems can achieve this:
  - Different prompts → diversity
  - Independent initial assessments → independence
  - Specialized roles → decentralized expertise
  - Analyst synthesis or voting → aggregation

### Debate Improves Accuracy

- Research shows LLMs are more accurate when they "debate" solutions
- Debate surfaces hidden assumptions and errors
- Multiple rounds allow error correction
- Connection to game theory: Debate is a **signaling game** where agents reveal information

## Practical Considerations

### Cost Management

- Multi-agent systems make many API calls
- Example: 4 agents, 3 rounds, 2 messages per round = 24 API calls
- At $0.003 per 1K input tokens, $0.015 per 1K output tokens (Claude Sonnet):
  - Average message: ~500 input tokens, ~200 output tokens
  - Cost per message: ~$1.50 + ~$3.00 = $4.50 per 1000 messages
  - Our example: ~$0.11 per scenario

- Strategies to reduce cost:
  - Use smaller/cheaper models for simple agents
  - Cache common responses
  - Limit conversation rounds
  - Implement early stopping when consensus reached

### Conversation Management

- **Infinite loops**: Agents might talk forever without reaching decision
  - Solution: Maximum turn limits, explicit termination conditions
- **Off-topic drift**: Agents might lose focus on original question
  - Solution: Regular reminders in prompts, moderator agent
- **Echo chambers**: Agents might all converge to same view prematurely
  - Solution: Explicit "devil's advocate" role, diversity requirements

### Quality Control

- How do we know if multi-agent outputs are good?
- Evaluation approaches:
  1. **Human review**: Gold standard but expensive
  2. **Automated metrics**: Consistency checks, format validation
  3. **Benchmark tasks**: Test on known problems
  4. **A/B testing**: Compare multi-agent vs. single-agent outputs

## Exercises

### Exercise 1: Extend the Disaster Response Team

Add a fifth agent to the disaster response team: a **Resource Allocation Economist**

Tasks:
1. Write an appropriate system prompt for this agent
2. Consider: cost-benefit analysis, opportunity cost, resource constraints
3. Re-run the disaster scenario with this new agent
4. Observe: How does the additional perspective change the recommendations?
5. Discuss: Is more agents always better? What are the tradeoffs?

In [9]:
# TODO: Create the economist agent and re-run the scenario

economist = AIAgent(
    "Economist",
    """TODO: Your system prompt here"""
)

# TODO: Create new team including economist
# TODO: Run disaster_response_round with new team

AIAgent(role="Economist", system_prompt="TODO: Your system prompt here", model="claude-haiku-4-5", memory=0 messages, api_key="***L0gAA")

### Exercise 2: Game-Playing Agents

Create two agents that play the Prisoner's Dilemma (from Week 8)

Setup:
- Two agents: "Player1" and "Player2"
- Each must choose: Cooperate or Defect
- Payoffs as in Week 8: (C,C)=(-1,-1), (C,D)=(-10,0), (D,C)=(0,-10), (D,D)=(-4,-4)

Tasks:
1. Create agent prompts that explain the game and payoffs
2. Have agents choose actions simultaneously (without seeing other's choice)
3. Reveal outcomes and payoffs
4. Run for 5 rounds - do agents learn to cooperate or defect?
5. Compare to Nash equilibrium prediction from game theory

Advanced:
- Allow agents to send messages before deciding
- Does communication enable cooperation?
- Connection: This is **cheap talk** in game theory

In [ ]:
# TODO: Implement Prisoner's Dilemma with AI agents

player1 = AIAgent(
    "Player1",
    """TODO: Your prompt here - explain the game, ask for decision"""
)

# TODO: Create player2
# TODO: Implement game loop
# TODO: Track and display results

### Exercise 3: Network Communication Patterns

Implement different communication topologies for multi-agent systems

Given: 5 agents working on a problem (e.g., writing a research paper)

Topologies to implement:
1. **Chain**: Agent 1 → Agent 2 → Agent 3 → Agent 4 → Agent 5
2. **Star**: Agent 1 (hub) communicates with all others
3. **Fully connected**: Every agent can talk to every other agent

Tasks:
1. Implement each topology
2. Give all agents the same task (e.g., "Write an abstract for a paper on network effects in social media")
3. Compare the outputs and efficiency (number of API calls, time to completion)
4. Analyze: Which topology produces best output? Which is most efficient?

Connection to Week 3-4: This explores **network structure** effects on information flow

In [ ]:
# TODO: Implement different network topologies

function chain_topology(agents::Vector{AIAgent}, task::String)
    # TODO: Sequential communication
end

function star_topology(agents::Vector{AIAgent}, task::String)
    # TODO: Hub-and-spoke communication
end

function fully_connected_topology(agents::Vector{AIAgent}, task::String)
    # TODO: All-to-all communication
end

# TODO: Create 5 agents with different writing specialties
# TODO: Run each topology and compare

### Exercise 4: Consensus vs. Voting Analysis

Compare the voting and consensus mechanisms empirically

Scenario: A university department must decide on a new curriculum

Agents (faculty members with different priorities):
- Theory-focused researcher
- Industry-oriented professor  
- Student-experience advocate
- Diversity and inclusion champion

Tasks:
1. Create the four agents with appropriate prompts
2. Present a curriculum decision (e.g., "Should we require a data ethics course?")
3. Run both the voting and consensus mechanisms
4. Compare:
   - Time to decision (API calls)
   - Quality of reasoning
   - Minority viewpoints (are they heard?)
   - Satisfaction (would agents support final decision?)
5. Discuss: When is each mechanism appropriate?

In [ ]:
# TODO: Create faculty agents and compare decision mechanisms

## Key Takeaways

### Multi-Agent Systems

- Multiple AI agents can collaborate to solve problems beyond individual capabilities
- **Specialization** allows agents to develop domain expertise
- **Communication** enables coordination without central control
- **Emergence** produces solutions not explicitly programmed

### Connection to Course Themes

- **Networks**: Agent communication forms directed graphs with different topologies
- **Game Theory**: Multi-agent interactions are strategic games with equilibria
- **ABMs**: AI agents extend traditional ABMs with reasoning and learning
- **Complexity**: Simple rules (prompts) + interaction → complex adaptive systems

### Design Patterns

- **Sequential coordination**: Chain of specialists (assembly line)
- **Debate**: Multiple perspectives with synthesis
- **Voting**: Democratic aggregation of preferences
- **Consensus**: Discussion until agreement
- **Hierarchical**: Managers and workers with different roles

### Practical Implications

- Multi-agent systems are powerful but expensive (many API calls)
- Need careful design to avoid infinite loops and off-topic drift
- Quality control requires evaluation against benchmarks
- Real-world applications: software development, research, decision support, crisis response

### Looking Ahead

- Next week (A2): We'll add **tool use** - agents that can execute code and access data
- We'll explore **PydanticAI patterns** for type-safe agent development
- We'll build agents that can analyze networks and generate reports
- The combination of multi-agent collaboration + tool use = **agentic workflows**

## Further Reading

### Multi-Agent Systems
- [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Communicative Agents for Software Development](https://arxiv.org/abs/2307.07924)
- [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442)

### Collective Intelligence
- [Improving Factuality and Reasoning in Language Models through Multiagent Debate](https://arxiv.org/abs/2305.14325)
- [Constitutional AI: Harmlessness from AI Feedback](https://arxiv.org/abs/2212.08073)
- Surowiecki, J. (2004). *The Wisdom of Crowds*

### Game Theory and AI
- [Strategic Reasoning with Language Models](https://arxiv.org/abs/2305.19165)
- [Emergent Communication in Multi-Agent Reinforcement Learning](https://arxiv.org/abs/2006.02419)

### Real-World Applications
- [Multi-Agent Collaboration for Disaster Response](https://www.google.com/about/stories/area-120-disaster-response/) (Inspiration for our case study)
- [GitHub Copilot Workspace](https://github.blog/2024-04-29-github-copilot-workspace/) (Multi-agent coding assistant)